In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models

import cv2
import numpy as np
from PIL import Image

import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score

from tqdm import tqdm

import copy

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [9]:
VIDEO_DIR = "/content/drive/MyDrive/videos_basketball"
FRAME_DIR = '/content/drive/MyDrive/ExtractedFrames'
os.makedirs(FRAME_DIR, exist_ok=True)

In [10]:
# Extract frames from videos
def extract_frames(video_dir, output_dir, max_frames=30):
    for file in os.listdir(video_dir):
        if file.endswith('.mp4'):
            label = 'hit' if 'hit' in file.lower() else 'miss'
            video_path = os.path.join(video_dir, file)
            video_name = file.split('.')[0]
            save_folder = os.path.join(output_dir, f"{label}_{video_name}")
            os.makedirs(save_folder, exist_ok=True)

            cap = cv2.VideoCapture(video_path)
            count = 0
            while count < max_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame_path = os.path.join(save_folder, f"frame_{count:04d}.jpg")
                cv2.imwrite(frame_path, frame)
                count += 1
            cap.release()

extract_frames(VIDEO_DIR, FRAME_DIR)

In [30]:
#  Dataset + Dataloader (Only Train + Test)
class FrameDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

def load_dataset(frame_dir):
    paths, labels = [], []
    for folder in os.listdir(frame_dir):
        full = os.path.join(frame_dir, folder)
        label = 1 if 'hit' in folder.lower() else 0
        if os.path.isdir(full):
            for f in os.listdir(full):
                if f.endswith('.jpg'):
                    paths.append(os.path.join(full, f))
                    labels.append(label)
    return paths, labels

def create_dataloaders(batch_size=32):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    paths, labels = load_dataset(FRAME_DIR)

    # Split data into 70% train, 30% test
    X_train, X_test, y_train, y_test = train_test_split(
        paths, labels, test_size=0.3, random_state=42, stratify=labels
    )

    train_ds = FrameDataset(X_train, y_train, transform)
    test_ds  = FrameDataset(X_test,  y_test,  transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, test_loader

In [14]:
#  Dataset + Dataloader (Only Train + Val)
class FrameDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

def load_dataset(frame_dir):
    paths, labels = [], []
    for folder in os.listdir(frame_dir):
        full = os.path.join(frame_dir, folder)
        label = 1 if 'hit' in folder.lower() else 0
        if os.path.isdir(full):
            for f in os.listdir(full):
                if f.endswith('.jpg'):
                    paths.append(os.path.join(full, f))
                    labels.append(label)
    return paths, labels

# Ici on ajoute batch_size en paramètre avec une valeur par défaut
def create_dataloaders(batch_size=32):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
    ])
    paths, labels = load_dataset(FRAME_DIR)
    X_train, X_val, y_train, y_val = train_test_split(
        paths, labels, test_size=0.2, random_state=42
    )
    train_ds = FrameDataset(X_train, y_train, transform)
    val_ds   = FrameDataset(X_val,   y_val,   transform)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)
    return train_loader, val_loader

In [31]:
# Train and test the finetuned Resnet Model
def train_resnet(num_epochs=10, lr=1e-3, batch_size=16):
    train_loader, test_loader = create_dataloaders(batch_size=batch_size)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Using device:", device)

    # Charger ResNet50 pré-entraîné
    model = models.resnet50(pretrained=True)
    for p in model.parameters():
        p.requires_grad = True  # Fine-tune toutes les couches
    model.fc = nn.Linear(model.fc.in_features, 2)  # Adapter la tête à 2 classes
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0
        for X, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
        train_acc = 100 * correct / total
        print(f"Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    #Évaluation finale sur le test set
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    test_acc = 100 * correct / total
    print(f"\nFinal Test Accuracy: {test_acc:.2f}%")

    # Sauvegarde du modèle entraîné
    torch.save(model.state_dict(), "/content/drive/MyDrive/best_resnet50_test.pth")
    print("Final model saved to Google Drive.")

In [32]:
train_resnet(num_epochs=10, lr=0.001, batch_size=16)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/10: 100%|██████████| 212/212 [00:59<00:00,  3.54it/s]


Train Loss: 0.2712, Train Acc: 90.23%


Epoch 2/10: 100%|██████████| 212/212 [01:00<00:00,  3.49it/s]


Train Loss: 0.0907, Train Acc: 96.37%


...

Epoch 9/10: 100%|██████████| 212/212 [00:59<00:00,  3.55it/s]


Train Loss: 0.0289, Train Acc: 98.11%


Epoch 10/10: 100%|██████████| 212/212 [01:01<00:00,  3.46it/s]

Train Loss: 0.0285, Train Acc: 98.17%



Final Test Accuracy: 98.35%
Final model saved to Google Drive.


In [15]:
# Entraînement et fine-tuning ResNet50
def train_resnet(num_epochs=10, lr=1e-4, batch_size=32):
    train_loader, val_loader = create_dataloaders(batch_size=batch_size)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Using device:", device)

    model = models.resnet50(pretrained=True)
    for p in model.parameters():
        p.requires_grad = True
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    best_acc = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0
        for X, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            preds = out.argmax(dim=1)
            total += y.size(0)
            correct += (preds == y).sum().item()
        print(f"Train Loss: {running_loss/len(train_loader):.4f}, Acc: {100*correct/total:.2f}%")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                out = model(X)
                preds = out.argmax(dim=1)
                total += y.size(0)
                correct += (preds == y).sum().item()
        val_acc = 100 * correct / total
        print(f"Val Acc: {val_acc:.2f}%")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "/content/drive/MyDrive/best_resnet50.pth")
            print("Best ResNet model saved.")

# Lancer l’entraînement (batch_size et autres hyperparamètres configurables)
train_resnet(num_epochs=10, lr=1e-4, batch_size=32)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 206MB/s]
Epoch 1/10: 100%|██████████| 122/122 [01:06<00:00,  1.83it/s]

Train Loss: 0.0674, Acc: 97.19%


Val Acc: 98.56%
✅ Best ResNet model saved.


Epoch 2/10: 100%|██████████| 122/122 [01:06<00:00,  1.82it/s]

...

Train Loss: 0.0276, Acc: 98.27%


Val Acc: 98.35%


Epoch 10/10: 100%|██████████| 122/122 [01:06<00:00,  1.84it/s]

Train Loss: 0.0279, Acc: 98.27%


Val Acc: 98.35%


### Tuning des paramètres de Resnet

In [17]:
def run_resnet_experiment(hparams):
    # Préparer les loaders avec batch_size variable
    train_loader, val_loader = create_dataloaders(batch_size=hparams['batch_size'])

    # Initialiser le modèle
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = models.resnet50(pretrained=True)

    # On fine-tune toutes les couches
    for p in model.parameters():
        p.requires_grad = True

    # On adapte la tête
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    # Définir perte et optimiseur avec weight decay
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=hparams['lr'],
        weight_decay=hparams['weight_decay']
    )

    best_acc = 0.0

    # Boucle d'entraînement + validation
    for epoch in range(hparams['num_epochs']):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                preds = model(X).argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)
        val_acc = 100 * correct / total
        best_acc = max(best_acc, val_acc)
        print(f"Epoch {epoch+1}/{hparams['num_epochs']} — Val Acc: {val_acc:.2f}%")

    return best_acc

In [18]:
hparam_space = [
    {'lr': 1e-3, 'batch_size': 16, 'weight_decay': 0.0,  'num_epochs': 5},
    {'lr': 1e-3, 'batch_size': 32, 'weight_decay': 1e-4, 'num_epochs': 5},
    {'lr': 1e-4, 'batch_size': 32, 'weight_decay': 0.0,  'num_epochs': 5},
    {'lr': 1e-4, 'batch_size': 64, 'weight_decay': 1e-4, 'num_epochs': 5},
]

In [19]:
results = []
for hparams in hparam_space:
    print(f"\nTesting config: {hparams}")
    acc = run_resnet_experiment(hparams)
    results.append((hparams, acc))

# Trier et afficher la meilleure config
results.sort(key=lambda x: x[1], reverse=True)
best_hparams, best_acc = results[0]
print("\nBest ResNet50 config:")
print(f"   Hyperparams: {best_hparams}")
print(f"   Validation Accuracy: {best_acc:.2f}%")


Testing config: {'lr': 0.001, 'batch_size': 16, 'weight_decay': 0.0, 'num_epochs': 5}


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5 — Val Acc: 98.35%
Epoch 2/5 — Val Acc: 98.35%
Epoch 3/5 — Val Acc: 98.56%
Epoch 4/5 — Val Acc: 98.56%
Epoch 5/5 — Val Acc: 98.56%

Testing config: {'lr': 0.001, 'batch_size': 32, 'weight_decay': 0.0001, 'num_epochs': 5}
Epoch 1/5 — Val Acc: 63.98%
Epoch 2/5 — Val Acc: 84.52%
Epoch 3/5 — Val Acc: 98.56%
Epoch 4/5 — Val Acc: 98.56%
Epoch 5/5 — Val Acc: 98.56%

Testing config: {'lr': 0.0001, 'batch_size': 32, 'weight_decay': 0.0, 'num_epochs': 5}
Epoch 1/5 — Val Acc: 98.56%
Epoch 2/5 — Val Acc: 98.35%
Epoch 3/5 — Val Acc: 98.56%
Epoch 4/5 — Val Acc: 98.56%
Epoch 5/5 — Val Acc: 98.56%

Testing config: {'lr': 0.0001, 'batch_size': 64, 'weight_decay': 0.0001, 'num_epochs': 5}
Epoch 1/5 — Val Acc: 98.35%
Epoch 2/5 — Val Acc: 98.56%
Epoch 3/5 — Val Acc: 98.56%
Epoch 4/5 — Val Acc: 98.35%
Epoch 5/5 — Val Acc: 98.56%

Best ResNet50 config:
   Hyperparams: {'lr': 0.001, 'batch_size': 16, 'weight_decay': 0.0, 'num_epochs': 5}
   Validation Accuracy: 98.56%


In [20]:
train_resnet(num_epochs=10, lr=1e-3, batch_size=16)

Using device: cuda


Epoch 1/10: 100%|██████████| 243/243 [01:09<00:00,  3.51it/s]

Train Loss: 0.2567, Acc: 91.17%


Val Acc: 83.49%
✅ Best ResNet model saved.


Epoch 2/10: 100%|██████████| 243/243 [01:10<00:00,  3.46it/s]

...

Train Loss: 0.1387, Acc: 95.40%


Val Acc: 98.25%


Epoch 10/10: 100%|██████████| 243/243 [01:08<00:00,  3.55it/s]

Train Loss: 0.0636, Acc: 97.93%


Val Acc: 98.56%


In [33]:
# Train and test a simple CNN
def train_simple_cnn(num_epochs=10, lr=1e-3, batch_size=16, num_filters=32, dropout=0.5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training SimpleCNN with: epochs={num_epochs}, lr={lr}, batch_size={batch_size}")

    # Utiliser le DataLoader avec split 70/30
    train_loader, test_loader = create_dataloaders_custom(batch_size)

    # Créer le modèle
    model = SimpleCNN(num_filters=num_filters, dropout=dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Entraînement
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
        acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Train Loss={running_loss/len(train_loader):.4f}, Train Acc={acc:.2f}%")

    # Évaluation finale sur test set
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            preds = model(X).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    test_acc = 100 * correct / total
    print(f"\nFinal Test Accuracy (SimpleCNN): {test_acc:.2f}%")

    # Sauvegarde
    torch.save(model.state_dict(), "/content/drive/MyDrive/best_simple_cnn_test.pth")
    print("SimpleCNN model saved.")


In [34]:
#Tester le cnn simple entrainé
train_simple_cnn(
    num_epochs=10,
    lr=0.001,
    batch_size=16,
    num_filters=32,
    dropout=0.5
)

Training SimpleCNN with: epochs=10, lr=0.001, batch_size=16
Epoch 1: Train Loss=0.3127, Train Acc=95.43%
Epoch 2: Train Loss=0.0343, Train Acc=98.30%
Epoch 3: Train Loss=0.0296, Train Acc=98.19%
Epoch 4: Train Loss=0.0292, Train Acc=98.24%
Epoch 5: Train Loss=0.0285, Train Acc=98.27%
Epoch 6: Train Loss=0.0320, Train Acc=98.30%
Epoch 7: Train Loss=0.0653, Train Acc=97.83%
Epoch 8: Train Loss=0.0300, Train Acc=98.37%
Epoch 9: Train Loss=0.0556, Train Acc=98.01%
Epoch 10: Train Loss=0.0338, Train Acc=98.37%

Final Test Accuracy (SimpleCNN): 98.56%
SimpleCNN model saved.


In [23]:
# CNN personnalisé
class SimpleCNN(nn.Module):
    def __init__(self, num_filters=32, dropout=0.5):
        super().__init__()
        self.conv1    = nn.Conv2d(3, num_filters, kernel_size=3, padding=1)
        self.pool     = nn.MaxPool2d(2,2)
        self.conv2    = nn.Conv2d(num_filters, num_filters*2, kernel_size=3, padding=1)
        self.fc1      = nn.Linear((224//4)*(224//4)*(num_filters*2), 128)
        self.dropout  = nn.Dropout(dropout)
        self.fc2      = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # → [B, N, 112,112]
        x = self.pool(F.relu(self.conv2(x)))  # → [B, 2N,  56,56]
        x = x.view(x.size(0), -1)             # flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

def create_dataloaders_custom(batch_size):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    paths, labels = load_dataset(FRAME_DIR)
    X_train, X_val, y_train, y_val = train_test_split(paths, labels, test_size=0.2, random_state=42)
    train_ds = FrameDataset(X_train, y_train, transform)
    val_ds   = FrameDataset(X_val,   y_val,   transform)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)
    return train_loader, val_loader

In [24]:
# Expérience pour un set d'hyperparamètres
def run_experiment(hparams):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Testing: {hparams}")
    train_loader, val_loader = create_dataloaders_custom(hparams['batch_size'])
    model = SimpleCNN(num_filters=hparams['num_filters'], dropout=hparams['dropout']).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=hparams['lr'])

    best_acc = 0
    for epoch in range(hparams['num_epochs']):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                preds = model(X).argmax(dim=1)
                correct += (preds==y).sum().item()
                total += y.size(0)
        val_acc = 100 * correct / total
        best_acc = max(best_acc, val_acc)
        print(f"  Epoch {epoch+1}: Val Acc={val_acc:.2f}%")
    return best_acc

In [25]:
hparams_simple = {
    'lr': 1e-3,
    'batch_size': 16,
    'num_filters': 32,
    'dropout': 0.5,
    'num_epochs': 10
}

simple_acc = run_experiment(hparams_simple)

print(f"\nSimpleCNN validation accuracy: {simple_acc:.2f}%")

Testing: {'lr': 0.001, 'batch_size': 16, 'num_filters': 32, 'dropout': 0.5, 'num_epochs': 10}
  Epoch 1: Val Acc=98.35%
  Epoch 2: Val Acc=98.35%
  Epoch 3: Val Acc=98.56%
  Epoch 4: Val Acc=98.56%
  Epoch 5: Val Acc=98.56%
  Epoch 6: Val Acc=98.25%
  Epoch 7: Val Acc=98.56%
  Epoch 8: Val Acc=98.56%
  Epoch 9: Val Acc=98.56%
  Epoch 10: Val Acc=98.56%

SimpleCNN validation accuracy: 98.56%


In [35]:
def run_experiment(hparams):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nTesting config: {hparams}")

    train_loader, test_loader = create_dataloaders_custom(hparams['batch_size'])
    model = SimpleCNN(num_filters=hparams['num_filters'], dropout=hparams['dropout']).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=hparams['lr'])

    for epoch in range(hparams['num_epochs']):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

    # Évaluation sur le test set
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            preds = model(X).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    test_acc = 100 * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")
    return test_acc


In [36]:
#Recherche sur grille
hparam_space = [
    {'lr':1e-3, 'batch_size':32, 'num_filters':32, 'dropout':0.3, 'num_epochs':5},
    {'lr':1e-4, 'batch_size':64, 'num_filters':64, 'dropout':0.5, 'num_epochs':5},
    {'lr':5e-4, 'batch_size':32, 'num_filters':64, 'dropout':0.3, 'num_epochs':5},
]

results = []
for h in hparam_space:
    acc = run_experiment(h)
    results.append((h, acc))

# Affichage du meilleur
results.sort(key=lambda x: x[1], reverse=True)
print("\nBest config:", results[0])


Testing config: {'lr': 0.001, 'batch_size': 32, 'num_filters': 32, 'dropout': 0.3, 'num_epochs': 5}
Test Accuracy: 98.56%

Testing config: {'lr': 0.0001, 'batch_size': 64, 'num_filters': 64, 'dropout': 0.5, 'num_epochs': 5}
Test Accuracy: 98.04%

Testing config: {'lr': 0.0005, 'batch_size': 32, 'num_filters': 64, 'dropout': 0.3, 'num_epochs': 5}
Test Accuracy: 98.35%

Best config: ({'lr': 0.001, 'batch_size': 32, 'num_filters': 32, 'dropout': 0.3, 'num_epochs': 5}, 98.55521155830753)


In [38]:
def evaluate_metrics(model, test_loader, model_name="Model"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds)

    print(f"\nEvaluation of {model_name}:")
    print(f"Accuracy : {acc*100:.2f}%")
    print(f"Recall   : {rec*100:.2f}%")
    print(f"F1-score : {f1*100:.2f}%")

    return acc, rec, f1


In [39]:
train_loader, test_loader = create_dataloaders(batch_size=16)

resnet = models.resnet50(pretrained=False)
resnet.fc = nn.Linear(resnet.fc.in_features, 2)
resnet.load_state_dict(torch.load("/content/drive/MyDrive/best_resnet50_test.pth"))

evaluate_metrics(resnet, test_loader, model_name="ResNet50")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



Evaluation of ResNet50:
Accuracy : 98.35%
Recall   : 98.51%
F1-score : 98.94%


(0.9834824501032347, 0.9850615114235501, 0.9894086496028244)

In [40]:
train_loader, test_loader = create_dataloaders_custom(batch_size=16)

simple_cnn = SimpleCNN(num_filters=32, dropout=0.5)
simple_cnn.load_state_dict(torch.load("/content/drive/MyDrive/best_simple_cnn_test.pth"))

evaluate_metrics(simple_cnn, test_loader, model_name="SimpleCNN")


Evaluation of SimpleCNN:
Accuracy : 98.56%
Recall   : 100.00%
F1-score : 99.09%


(0.9855521155830753, 1.0, 0.9908854166666666)